In [1]:
BERT_VOCAB= '../uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '../uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '../uncased_L-12_H-768_A-12/bert_config.json'

In [2]:
##install bert if not already done
!pip install bert-tensorflow

In [3]:
import os
import numpy as np
import nltk
import pandas as pd
import bs4
import re
import bert
import datetime as dt
import pickle

from bs4 import BeautifulSoup
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from datetime import datetime
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USCHAS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USCHAS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
DATA_PATH = '../data/'
NON_SPAM_TEST = DATA_PATH +'nonspam-test/'
NON_SPAM_TRAIN = DATA_PATH +'nonspam-train/'
SPAM_TEST = DATA_PATH +'spam-test/'
SPAM_TRAIN = DATA_PATH +'spam-test/'

In [6]:
non_spam_test_list = os.listdir(NON_SPAM_TEST)
non_spam_train_list = os.listdir(NON_SPAM_TRAIN)
spam_test_list = os.listdir(SPAM_TEST)
spam_train_list = os.listdir(SPAM_TRAIN)

In [7]:
df =pd.DataFrame(columns=['id','text','label','split'])

In [8]:
for filename in non_spam_test_list:
    with open(NON_SPAM_TEST+filename,"r") as fd:
        text = fd.read()
        ID=filename.split(".")[0]
        label='no'
        split='test'
        df.loc[df.shape[0]]=[ID]+[text]+[label]+[split]

In [9]:
for filename in non_spam_train_list:
    with open(NON_SPAM_TRAIN+filename,"r") as fd:
        text = fd.read()
        ID=filename.split(".")[0]
        label='no'
        split='train'
        df.loc[df.shape[0]]=[ID]+[text]+[label]+[split]

In [10]:
for filename in spam_train_list:
    with open(SPAM_TRAIN+filename,"r") as fd:
        text = fd.read()
        ID=filename.split(".")[0]
        label='yes'
        split='train'
        df.loc[df.shape[0]]=[ID]+[text]+[label]+[split]

In [11]:
for filename in spam_test_list:
    with open(SPAM_TEST+filename,"r") as fd:
        text = fd.read()
        ID=filename.split(".")[0]
        label='yes'
        split='test'
        df.loc[df.shape[0]]=[ID]+[text]+[label]+[split]

In [12]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
bert_tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)
bert_tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [13]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sno = nltk.stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
from tqdm import tqdm
preprocessed_synop = []
for sentance in tqdm(df['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    stemmed_sentence = []
    for e in sentance.split():
        if e.lower() not in stopwords:
            s=(sno.stem(lemmatizer.lemmatize(e.lower()))).encode('utf8')#lemitizing and stemming each word
            stemmed_sentence.append(s)
    sentance = b' '.join(stemmed_sentence)
    preprocessed_synop.append(sentance)
    
df['CleanedSynopsis']=preprocessed_synop #adding a column of CleanedText which displays the data after pre-processing of the review 
df['CleanedSynopsis']=df['CleanedSynopsis'].str.decode("utf-8")

100%|██████████| 740/740 [00:11<00:00, 63.14it/s]


In [14]:
x = df['CleanedSynopsis']
Y = df['label']

y=np.zeros((Y.shape[0],1))

for i in range(len(y)):
    if(Y[i]=='no'):
        y[i]=1
    else:
        y[i]=0

In [15]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [16]:
y_train_vect = yTrain
y_test_vect = yTest

In [23]:
start = datetime.now()

vectorizer = TfidfVectorizer(min_df=10,max_features=20000,smooth_idf=True,norm="l2",\
                              tokenizer=lambda x:bert_tokenizer.tokenize(x),sublinear_tf=False,ngram_range=(1,4))
x_train_ngram = vectorizer.fit_transform(xTrain)
x_test_ngram = vectorizer.transform(xTest)
print("Time Taken to run this cell:, ", datetime.now() - start)

Time Taken to run this cell:,  0:00:07.203728


In [18]:
clf = MultinomialNB()
clf.fit(x_train_ngram, y_train_vect)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

C:\Users\USCHAS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
y_pred = clf.predict(x_test_ngram)
accuracy_score(y_test_vect, y_pred, normalize=True)

0.972972972972973

In [20]:
out ='../output/'
filename = 'finalized_model.sav'
pickle.dump(clf, open(out+filename, 'wb'))

In [27]:
pickle.dump(xTrain, open(out + 'xTrain.pk', 'wb'))